# Named entity recognition

## Load ALTO

In [1]:
from lxml import etree
import sys

def alto_parse(alto):
    # from: https://github.com/cneud/alto-tools/blob/master/alto_tools.py
    """ Convert ALTO xml file to element tree """
    try:
        xml = etree.parse(alto)
    except etree.ParseError as e:
        sys.stdout.write('\nERROR: Failed parsing "%s" - '
                         % alto.name + str(e))
    # http://lxml.de/tutorial.html#namespaces
    # Register ALTO namespaces
    namespace = {'alto-1': 'http://schema.ccs-gmbh.com/ALTO',
                 'alto-2': 'http://www.loc.gov/standards/alto/ns-v2#',
                 'alto-3': 'http://www.loc.gov/standards/alto/ns-v3#',
                 'alto-4': 'http://www.loc.gov/standards/alto/ns-v4#'}
    # Extract namespace from document root
    if 'http://' in str(xml.getroot().tag.split('}')[0].strip('{')):
        xmlns = xml.getroot().tag.split('}')[0].strip('{')
    else:
        try:
            ns = xml.getroot().attrib
            xmlns = str(ns).split(' ')[1].strip('}').strip("'")
        except IndexError:
            sys.stdout.write('\nWARNING: File "%s": no namespace declaration '
                             'found.' % alto.name)
            xmlns = 'no_namespace_found'
    if xmlns in namespace.values():
        return alto, xml, xmlns
    else:
        sys.stdout.write('\nWARNING: File "%s": namespace is not registered.'
                         % alto.name)

In [2]:
from glob import glob

fns = sorted(list(glob('../data/Nieuws_Advertentieblad_XmlOnly/*/*/alto/*.xml')))
print(len(fns))
print(fns[:3])

10753
['../data/Nieuws_Advertentieblad_XmlOnly/1876/01/alto/CDWd0001_1876_01_01.xml', '../data/Nieuws_Advertentieblad_XmlOnly/1876/01/alto/CDWd0001_1876_01_02.xml', '../data/Nieuws_Advertentieblad_XmlOnly/1876/01/alto/CDWd0001_1876_01_03.xml']


In [3]:
fn = fns[99]
_, tree, xmlns = alto_parse(fn)

In [4]:
def alto_text(xml, xmlns):
    # edited from: https://github.com/cneud/alto-tools/blob/master/alto_tools.py
    """ Extract text content from ALTO xml file """
    # Find all TextLine elements
    text = ''
    for lines in xml.iterfind('.//{%s}TextLine' % xmlns):
        # New line after every TextLine element
        text += '\n'
        # Find all String elements
        for line in lines.findall('{%s}String' % xmlns):
            # Get value of attribute CONTENT from all String elements
             text += line.attrib.get('CONTENT') + ' '
    return text

In [5]:
text = alto_text(tree, xmlns)
print(text[:1000])


1 ° Meedeel ingen ;  
2 * Benoeming van drie leden der deputatie . 
De zitting wordt ten 1 1/4 ure geheven . 
Zitting van 6 juli . 
De zitting wordt geopend ten 10 1/2 ure met de naamoproe 
ping der leden . 
M . Reypens , sekrelaris , geeft lezing van het proces-verbaal der 
vorige zitting . Hetzelve wordt goedgekeurd . 
MEÊDEEL1NGEN . 
o . Bet gemeentebestuur van Ressel klaagt over het slecht 
onderhoud van eenen steenweg . 
M . VAN CATJWENBERGB vraagt de neerlegging op het bureel 
gedurende de beraadslaging over het budjet . Aangenomen . 
b . Verschillende gemeente-besturen vragen eenen steenweg 
van Viersel , langs Pulderbosch , naar Zoersel . 
M . ANTHONI vraagt de verzending naar de middensektie . 
Aangenomen . 
c . Ret gemeentebestuur van Gierle vraagt de stemming van 
een reglement om het gebruik van wielen met breede velgen 
voor te schrijven . 
MM . GILLES-ROSA en SMOLDEREN vragen de benoeming door 
het bureel . 
Deze kommissie wordt samengesteld uit MM . Sraólderen , Em . 
G

In [6]:
text = ' '.join(text.split())
print(text[:1000])

1 ° Meedeel ingen ; 2 * Benoeming van drie leden der deputatie . De zitting wordt ten 1 1/4 ure geheven . Zitting van 6 juli . De zitting wordt geopend ten 10 1/2 ure met de naamoproe ping der leden . M . Reypens , sekrelaris , geeft lezing van het proces-verbaal der vorige zitting . Hetzelve wordt goedgekeurd . MEÊDEEL1NGEN . o . Bet gemeentebestuur van Ressel klaagt over het slecht onderhoud van eenen steenweg . M . VAN CATJWENBERGB vraagt de neerlegging op het bureel gedurende de beraadslaging over het budjet . Aangenomen . b . Verschillende gemeente-besturen vragen eenen steenweg van Viersel , langs Pulderbosch , naar Zoersel . M . ANTHONI vraagt de verzending naar de middensektie . Aangenomen . c . Ret gemeentebestuur van Gierle vraagt de stemming van een reglement om het gebruik van wielen met breede velgen voor te schrijven . MM . GILLES-ROSA en SMOLDEREN vragen de benoeming door het bureel . Deze kommissie wordt samengesteld uit MM . Sraólderen , Em . Geelhand , Keysers , Gille

In [7]:
import spacy
nlp = spacy.load('nl_core_news_lg')

In [8]:
doc = nlp(text[:1000])

for token in doc:
    print(token.text, token.ent_type_)

1 CARDINAL
° 
Meedeel NORP
ingen 
; 
2 CARDINAL
* 
Benoeming 
van 
drie CARDINAL
leden 
der 
deputatie 
. 
De 
zitting 
wordt 
ten 
1 CARDINAL
1/4 CARDINAL
ure 
geheven 
. 
Zitting 
van 
6 DATE
juli DATE
. 
De 
zitting 
wordt 
geopend 
ten 
10 CARDINAL
1/2 CARDINAL
ure 
met 
de 
naamoproe 
ping 
der 
leden 
. 
M 
. 
Reypens PERSON
, 
sekrelaris PERSON
, 
geeft 
lezing 
van 
het 
proces-verbaal 
der 
vorige 
zitting 
. 
Hetzelve 
wordt 
goedgekeurd 
. 
MEÊDEEL1NGEN PERSON
. 
o 
. 
Bet 
gemeentebestuur 
van 
Ressel PERSON
klaagt 
over 
het 
slecht 
onderhoud 
van 
eenen 
steenweg 
. 
M 
. 
VAN 
CATJWENBERGB 
vraagt 
de 
neerlegging 
op 
het 
bureel 
gedurende 
de 
beraadslaging 
over 
het 
budjet 
. 
Aangenomen 
. 
b 
. 
Verschillende 
gemeente-besturen 
vragen 
eenen 
steenweg 
van 
Viersel GPE
, 
langs 
Pulderbosch LANGUAGE
, 
naar 
Zoersel GPE
. 
M 
. 
ANTHONI 
vraagt 
de 
verzending 
naar 
de 
middensektie 
. 
Aangenomen 
. 
c 
. 
Ret NORP
gemeentebestuur 
van 
Gierle 
vraagt 
de 
st

## Toponyms

In [10]:
import pandas as pd
locs = pd.read_excel('../data/ruimte/Output_ruimtelijke context_single_XLSX.xlsx')

In [11]:
locs.columns

Index(['IN_FID', 'IN_TOPON', 'IN_BRON', 'IN_TIMEFRAME', 'IN_CLASS', 'IN_Id',
       'IN_Opm', 'IN_POPULATIE', 'IN_X', 'IN_Y'],
      dtype='object')

In [12]:
names = [l.lower() for l in locs['IN_TOPON']]

In [14]:
print(len(names))
names[:120]

559


['aartselaar',
 'antwerpen',
 'arendonk',
 'balen',
 'berendrecht',
 'beerse',
 'berlaar',
 'berchem',
 'bevel',
 'borgerhout',
 'blaasveld',
 'boechout',
 'bonheiden',
 'boom',
 'bornem',
 'bouwel',
 'brasschaat',
 'borsbeek',
 'breendonk',
 'broechem',
 'burcht',
 'beerzel',
 'dessel',
 'deurne',
 'duffel',
 'edegem',
 'eindhout',
 'emblem',
 'essen',
 'gestel',
 'gierle',
 'grobbendonk',
 'halle',
 'heffen',
 'hemiksem',
 'herselt',
 'hallaar',
 'hoboken',
 'heist-op-den-berg',
 'hoevenen',
 'hombeek',
 'hoogstraten',
 'houtvenne',
 'hove',
 'herentals',
 'hulshout',
 'itegem',
 'kalmthout',
 'kapellen',
 'kasterlee',
 'kessel',
 'koningshooikt',
 'kontich',
 'leest',
 'lichtaart',
 'lier',
 'lille',
 'lint',
 'lillo',
 'lippelo',
 'liezele',
 'loenhout',
 'mariekerke',
 'massenhoven',
 'mechelen',
 'meer',
 'merksem',
 'mol',
 'mortsel',
 'meerhout',
 'morkhoven',
 'meerle',
 'merksplas',
 'niel',
 'nijlen',
 'noorderwijk',
 'oelegem',
 'olen',
 'olmen',
 'onze-lieve-vrouw-waver',


In [24]:
import os
import shutil

def clean_dir(d):
    try:
        shutil.rmtree(d)
    except FileNotFoundError:
        pass
    os.mkdir(d)

In [33]:
od = '../data/ner_out'
clean_dir(od)

In [34]:
import random
random.shuffle(fns)
fns_sample = fns[:500]

In [35]:
import tqdm
for fn in tqdm.tqdm(fns_sample):
    bn = os.path.basename(fn).replace('.xml', '')
    try:
        _, tree, xmlns = alto_parse(fn)
        text = alto_text(tree, xmlns)
    except:
        pass
    tokens = []
    for token in nlp(text):
        hit = False
        if token.text.lower() in names:
            hit = True
        tokens.append((token.text, token.ent_type_, hit))
    df = pd.DataFrame(tokens, columns=('token', 'entity', 'geo'))
    df.to_excel(f'{od}/{bn}.xlsx')

100%|██████████| 500/500 [11:32<00:00,  1.39s/it]


## Plot

In [ ]:
for fn in list(glob.glob(f'{d}/*.xlsx'))[:10]:
    print(f)
    loc = df.read_excel(fn)